> 합성곱

In [ ]:
import numpy as np
w = np.array([2, 1, 5, 3])  # 가중치, 필터, 커널
x = np.array([2, 8, 3, 7, 1, 2, 0, 4, 5])

w_r = np.flip(w)    #w 배열을 뒤집어서 출력
print(w_r)

In [ ]:
#합성곱 계산
# w_r을 x의 왼쪽 자리에 맞추고 각 인덱스마다 곱한 후 더함
# 2x3 + 8x5 + 3x1 + 7x2 = 63
# w_r을 오른쪽으로 한자리 shift하여 곱셈
for i in range(6):
    print(np.dot(x[i:i+4], w_r))

In [ ]:
#사이파이에서 제공하는 합성곱 함수
#w를 뒤집어서 곱하는 방식
from scipy.signal import convolve
# valid - 원본 배열에 패딩을 추가하지 않는 방식
# 원본 이미지가 4x4인 경우 결과물이 3x3으로 줄어드는 방식
convolve(x, w, mode='valid')

In [ ]:
#교차상관 - w를 뒤집지 않고 곱하는 방식
# '합성곱 신경망'에서는 w를 뒤집지 않고 그대로 곱하는 '교차상관' 방식을 사용함
# 초기 가중치값은 랜덤으로 만들어지므로 뒤집어서 곱하는 것과 
#뒤집지 않고 곱하는 것이 큰 의미가 없음
# 정확히 표현하면 교차상관이지만 합성곱 신경망이라는 이름을 관례적으로 사용하고 있음
from scipy.signal import correlate
correlate(x, w, mode='valid')

In [ ]:
#full 패딩 - 제로패딩을 한 후 연산을 하게 되면 
#원본 배열의 모든 원소가 연산에 동일하게 참여하게 됨
correlate(x, w, mode='full')

In [ ]:
#출력 배열의 길이가 원본 배열의 길이와 같아지도록 제로 패딩을 추가하는 방식
#합성곱 신경망에서 많이 사용하는 방식
correlate(x, w, mode='same')

In [ ]:
#2차원 배열에 대한 합성곱 계산
from scipy.signal import correlate2d
x = np.array([[1, 2, 3],
            [4, 5, 6],
            [7, 8, 9]])
w = np.array([[2, 0],
            [0, 0]])
correlate2d(x, w, mode='valid')

In [ ]:
#제로패딩을 하여 원본과 같은 사이즈로 출력되도록 함
correlate2d(x, w, mode='same')

In [ ]:
#텐서플로에서 지원하는 합성곱 함수
import tensorflow as tf

#4차원 배열을 사용해야 함
x = np.array([[1, 2, 3],
            [4, 5, 6],
            [7, 8, 9]])
with tf.device('/CPU:0'):
    # 입력값: reshape(batch, height, width, channel)
    x_4d = x.astype(np.float).reshape(1, 3, 3, 1) #실수형으로 입력해야 함
    # 필터(가중치) reshape(height,width,channel,가중치의개수)
    w_4d = w.reshape(2, 2, 1, 1)
    #SAME 대문자로 작성해야 함
    c_out = tf.nn.conv2d(x_4d, w_4d, strides=1, padding='SAME')
    # 텐서를 넘파이 배열로 변환
    print(c_out.numpy().reshape(3, 3))

In [ ]:
#맥스풀링
# 입력값: reshape(샘플수, height, width, channel)
x = np.array([[1, 2, 3, 4],
            [5, 6, 7, 8],
            [9, 10, 11, 12],
            [13, 14, 15, 16]])
x = x.reshape(1, 4, 4, 1)
# ksize 커널사이즈 2x2, strides 이동간격
with tf.device('/CPU:0'):
    p_out=tf.nn.max_pool2d(x,ksize=2,strides=2,padding='SAME')
    print(p_out.numpy().reshape(2,2))

> 한글텍스트 분류

In [1]:
# 메모리 부족할 때
# 메모리 필요한만큼만 사용하는 옵션
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [2]:
import pandas as pd
train_data = pd.read_csv('c:/vscode/data/text/ratings_train.csv',encoding='ms949')
test_data = pd.read_csv('c:/vscode/data/text/ratings_test.csv',encoding='ms949')

In [3]:
#중복값 제거
import numpy as np
train_data.drop_duplicates(subset=['document'], inplace=True)
# Null 값이 존재하는 행 제거
train_data = train_data.dropna(how = 'any')
#특수문자,기호 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
#공백 제거
train_data['document'] = train_data['document'].str.replace('^ +', "")
train_data['document'].replace('', np.nan, inplace=True)
#null 샘플 제거
train_data = train_data.dropna(how = 'any')

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6120\2266659944.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6120\2266659944.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "")


In [4]:
# test_data에도 동일한 과정 적용
test_data.drop_duplicates(subset = ['document'],inplace=True)
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['document'] = test_data['document'].str.replace('^ +', "")
test_data['document'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any')

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6120\4291877067.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6120\4291877067.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', "")


In [5]:
# 불용어 사전
stopwords = ['의','가','이','은','들','는','좀','잘',
            '걍','과','도','를','으로','자','에','와','한','하다']

In [6]:
from konlpy.tag import Okt
okt = Okt()
X_train = []
#형태소 분석
#for sentence in train_data['document']:
for sentence in train_data['document'][:10000]:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X 
            if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [7]:
# 테스트 데이터에 대한 토큰화
X_test = []
#for sentence in test_data['document']:
for sentence in test_data['document'][:10000]:
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X 
            if not word in stopwords]
    X_test.append(temp_X)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
# 정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
#print(tokenizer.word_index)

In [9]:
# 출현빈도가 3회 미만인 단어들
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어수
rare_cnt = 0
total_freq = 0
rare_freq = 0
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value
print(total_cnt) #단어집합 크기
print(rare_cnt) #희귀단어수

vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

12445
8200
단어 집합의 크기 : 4246


In [10]:
# 정수 인코딩
#텍스트를 숫자 시퀀스로 변환
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [11]:
y_train = np.array(train_data['label'][:10000])
y_test = np.array(test_data['label'][:10000])

max_len = max(len(l) for l in X_train) #리뷰의 최대 길이

cnt = 0
for s in X_train:
    if(len(s) <= 30):
        cnt = cnt + 1
#최대 길이 이하인 샘플의 비율
(cnt / len(X_train))*100

95.24000000000001

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Flatten, MaxPooling1D
from tensorflow.keras.models import load_model

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length = max_len))
model.add(Conv1D(filters = 64, kernel_size = 5, 
                padding = 'same',activation = 'relu', strides = 1))
model.add(Conv1D(filters = 32, kernel_size = 4, 
                padding = 'same',activation = 'relu', strides = 1))
model.add(Conv1D(filters = 16, kernel_size = 3, 
                padding = 'same',activation = 'relu', strides = 1))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', 
            optimizer ='adam', metrics=['acc'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 53, 100)           424600    
                                                                 
 conv1d (Conv1D)             (None, 53, 64)            32064     
                                                                 
 conv1d_1 (Conv1D)           (None, 53, 32)            8224      
                                                                 
 conv1d_2 (Conv1D)           (None, 53, 16)            1552      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 10, 16)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                        

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
mc = ModelCheckpoint('CNN_model.h5', monitor='val_acc', 
                    mode='max', verbose=1, save_best_only=True)
model.fit(X_train, y_train, batch_size = 64, epochs=10, 
        validation_split=0.2, callbacks=[es, mc])

Epoch 1/10
120/125 [===========================>..] - ETA: 0s - loss: 0.6163 - acc: 0.6236
Epoch 1: val_acc improved from -inf to 0.77500, saving model to CNN_model.h5
125/125 [==============================] - 7s 11ms/step - loss: 0.6087 - acc: 0.6305 - val_loss: 0.4744 - val_acc: 0.7750
Epoch 2/10
122/125 [============================>.] - ETA: 0s - loss: 0.3464 - acc: 0.8513
Epoch 2: val_acc improved from 0.77500 to 0.79150, saving model to CNN_model.h5
125/125 [==============================] - 1s 9ms/step - loss: 0.3472 - acc: 0.8505 - val_loss: 0.4455 - val_acc: 0.7915
Epoch 3/10
124/125 [============================>.] - ETA: 0s - loss: 0.2246 - acc: 0.9094
Epoch 3: val_acc did not improve from 0.79150
125/125 [==============================] - 1s 7ms/step - loss: 0.2255 - acc: 0.9093 - val_loss: 0.5288 - val_acc: 0.7805
Epoch 4/10
121/125 [============================>.] - ETA: 0s - loss: 0.1479 - acc: 0.9419
Epoch 4: val_acc did not improve from 0.79150
125/125 [==============

In [15]:
loaded_model = load_model('CNN_model.h5')
loaded_model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.4379 - acc: 0.7993


[0.43785560131073, 0.7993000149726868]

In [16]:
def review_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence 
                    if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) #정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(model.predict(pad_new)) # 예측
    if(score > 0.5):
        print(f"{score * 100:.2f}% 확률로 긍정 리뷰입니다.\n")
    else:
        print(f"{(1 - score) * 100:.2f}% 확률로 부정 리뷰입니다.\n")

In [17]:
review_predict('연기는 잔잔하게 볼 만 합니다')
review_predict('영화의 주제가 뭔지 모르겠음')
review_predict('익살스런 연기가 돋보였던 영화')
review_predict('기대보다는 스토리가 큰 감흥은 없습니다')

1/1 [==============================] - 0s 148ms/step
84.27% 확률로 긍정 리뷰입니다.

1/1 [==============================] - 0s 18ms/step
99.98% 확률로 부정 리뷰입니다.

1/1 [==============================] - 0s 19ms/step
99.82% 확률로 긍정 리뷰입니다.

1/1 [==============================] - 0s 19ms/step
100.00% 확률로 부정 리뷰입니다.

